<h1>SQL Select Queries</h1>
<p>The difficulty of these queries snippets have a varying degree of intricacy.<br></p>
<hr>

<h3>Query 1:</h3>
<p>
Skills shown:
<ul>
    <li>CTEs</li>
    <li>Partition by</li>
    <li>Aggregations</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>Calculate the month-over-month percentage change in revenue</p>
</p>
<p>Query:</p>



In [ ]:
with t2 as (select
            date_format(created_at,"%Y-%m") as date,
            sum(value) as revenue
            from sf_transactions
            group by date),
t3 as (select *, 
    lag(revenue) over(order by date) as last_month_rev
    from t2)
    
select date,
round(((revenue - last_month_rev)/last_month_rev)*100,2) as revenue_difference
from t3

Result:

<img src="Select_Queries_images//monthly_Percentage_Difference_h.PNG" alt="Result Image">

<hr>
<h3>Query 2</h3>
<p>
Skills shown:
<ul>
    <li>Joining multiple tables together</li>
    <li>Group by</li>
    <li>Aggregation with If statement</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>Calculate the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers.</p>
</p>
<p>Query:</p>


In [ ]:
select
    date,
    sum(IF(paying_customer =  "no",  downloads, 0)) as non_paying,
    sum(IF(paying_customer = "yes", downloads, 0)) as paying
from ms_user_dimension as us
join ms_download_facts as facts
    on facts.user_id = us.user_id
join ms_acc_dimension as acc
    on us.acc_id = acc.acc_id
group by date
having non_paying > paying
order by date asc

Result:<br>
<img src="Select_Queries_images//premium_vs_freemium_h.PNG" alt="Query Result" >

<hr>
<h3>Query 3</h3>
<p>
Skills shown:
<ul>
    <li>Union Function</li>
    <li>Sub-query</li>
    <li>Aggregation with If statement</li>
    <li>Group by/Order by functions</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>Find the popularity percentage for each user. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform</p>
</p>
<p>Query:</p>


In [ ]:
with friends as(select user1 from facebook_friends
                union all
                select user2 from facebook_friends)
select 
    user1,
    (count(user1)/((select count(distinct(user1)) as total_people 
        from friends)) * 100) as popularity
    from friends
    group by user1
    order by user1

Result(Limit of 5):<br>
<img src="Select_Queries_images//Popularity_Percentage_h.PNG" alt="Query result">

<hr>
<h3>Query 4</h3>
<p>
Skills shown:
<ul>
    <li>Union Function</li>
    <li>Like function/Wildcard Characters</li>
    <li>Sub-Query</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
    Find the number of times the words 'bull' and 'bear' occur in the contents<br>
    <em>Words like <strong>"bullish"</strong> should not be included in the count</em>
</p>
</p>
<p>Query:</p>

In [ ]:
select "bull" as word, (select count(contents) from google_file_store where lower(contents) like "% bull %" ) as count
union 
select "bear" as word, (select count(contents) from google_file_store where lower(contents) like "% bear %") as count

Result<br>
<img src="Select_Queries_images//Counting_instances_in_Text_h.PNG" alt="Query Result">

<hr>
<h3>Query 5</h3>
<p>
Skills shown:
<ul>
    <li>CTEs</li>
    <li>Case Function</li>
    <li>Aggregations</li>
    <li>Group by/Order by functions</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
    Find the minimum, average, maximum rental prices for each host’s popularity rating.<br>
    <em>There is no Unique id for Renters</em>
</p>
</p>
<p>Query:</p>

In [ ]:
with host_pop as(select CONCAT(price, room_type, host_since, zipcode, number_of_reviews) as host_id,
                    price,
                    case
                        when number_of_reviews = 0 then "New"
                        when number_of_reviews between 1 and 5 then "Rising"
                        when number_of_reviews between 6 and 15 then "Trending Up"
                        when number_of_reviews between 16 and 40 then "Popular"
                        when number_of_reviews > 40 then "Hot"
                    end as pop_rating
                    from airbnb_host_searches
                    group by host_id)

select 
    pop_rating,
    min(price) as min_price,
    avg(price) as average_price,
    max(price) as max_price
from host_pop
group by pop_rating
order by average_price desc

<img src="Select_Queries_images//Host_pop_rental_prices_h.PNG" alt="Query Result">

<hr>
<h3>Query 6</h3>
<p>
Skills shown:
<ul>
    <li>CTEs</li>
    <li>Aggregations/Window Functions</li>
    <li>Partition By</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
    Users that make their first in-app purchase are placed in a marketing campaign where they see call-to-actions for more in-app purchases. Find the number of users that made additional in-app purchases due to the success of the marketing campaign.<br><br>
    Case:<br>
    <em>The marketing campaign doesn't start until one day after the initial in-app purchase so users that only made one or multiple purchases on the first day do not count, nor do we count users that over time purchase only the products they purchased on the first day.</em>
</p>
</p>
<p>Query:</p>

In [ ]:
with sbqry as (select
                    user_id,
                    min(created_at) over(partition by user_id) as first_item_purchase_date,
                    min(created_at) over(partition by user_id, product_id) as item_purchase_date
                From marketing_campaign
                    )

select count(distinct(user_id)) as users
From sbqry
where item_purchase_date > first_item_purchase_date

Result:<br>
<strong>23</strong>

<hr>
<h3>Query 7</h3>
<p>
Skills shown:
<ul>
    <li>Window Function</li>
    <li>Where clause</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
     Identify returning active users<br><br>
     Case:<br>
     <em>A returning active user is a user that has made a second purchase within 7 days of any other of their purchases</em>
</p>
</p>
<p>Query:</p>

In [ ]:
select distinct(user_id)
from
    (select *, 
    lead(created_at) over(partition by user_id order by created_at) as following_purchase_date
    from amazon_transactions) as purchase_history
where abs(datediff(created_at,following_purchase_date)) <= 7

Result(limit of 6):<br>
<img src="Select_Queries_images//user_purchases_m.PNG" alt="Query Result">

<hr>
<h3>Query 8</h3>
<p>
Skills shown:
<ul>
    <li>CTEs</li>
    <li>Unions</li>
    <li>Aggregation function</li>
    <li>Where Clause</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
     Find the date with the highest total energy consumption from the data centers
</p>
</p>
<p>Query:</p>

In [ ]:
with all_energy 
as(
    select * 
    from fb_eu_energy as eu
    
    union all 
    
    select * 
    from fb_asia_energy
    
    union all
    
    select * 
    from fb_na_energy),
    
sum_energy 
as (
    select date, sum(consumption) as consumption
    from all_energy
    group by date)

select date, consumption
from sum_energy
where consumption in (select max(consumption) from sum_energy)


Result:<br>
<img src="Select_Queries_images//highest_energy_consumption_m.PNG" alt="Query Result">

<hr>
<h3>Query 9</h3>
<p>
Skills shown:
<ul>
    <li>CTE</li>
    <li>Window Function</li>
    <li>Partition by</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
     Find the employee with the highest salary per department.
</p>
</p>
<p>Query:</p>

In [ ]:
with ranked as (
    select
        department,
        first_name,
        salary,
        rank() over(partition by department order by salary desc) as dep_rank
        from employee
        )

select department,first_name,salary
from ranked
where dep_rank = 1

Result:<br>
<img src="Select_Queries_images//Highest_salary_Department_m.PNG" alt="Query Result">

<hr>
<h3>Query 10</h3>
<p>
Skills shown:
<ul>
    <li>Case Function</li>
    <li>Like/Regex </li>
    <li>Group by</li>
</ul>
</p>
<p>
<h5>Problem:</h5>
<p>
     Classify each business as either a restaurant, cafe, school, or other.
</p>
</p>
<p>Query:</p>

In [ ]:
select business_name,
    case
        when business_name like "%restaurant%" then "restaurant"
        when business_name like "%cafe%" or business_name like "%café%" or business_name like "%coffee%" then "cafe"
        when business_name like "%School%" then "school"
        else "other" 
        end as classification
from sf_restaurant_health_violations
group by business_name;

Result(Limit of 13):<br>
<img src="Select_Queries_images//Classify_bussiness_type_m.PNG" alt="Query Result">